In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch

In [ ]:
# ----------------------------
# Load dataset
# ----------------------------
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
# Map categories to ids
labels = sorted(train_df["Category"].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
train_df["label"] = train_df["Category"].map(label2id)
val_df["label"] = val_df["Category"].map(label2id)
test_df["label"] = test_df["Category"].map(label2id)

In [ ]:
# Hugging Face datasets
train_ds = Dataset.from_pandas(train_df[["Raw Input", "label"]])
val_ds = Dataset.from_pandas(val_df[["Raw Input", "label"]])
test_ds = Dataset.from_pandas(test_df[["Raw Input", "label"]])

In [ ]:
# ----------------------------
# Tokenizer & Model
# ----------------------------
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

def tokenize(batch):
    return tokenizer(batch["Raw Input"], padding="max_length", truncation=True, max_length=128)

In [ ]:
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(labels), id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ----------------------------
# Training
# ----------------------------
args = TrainingArguments(
    output_dir="./bert-spend-cls",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [ ]:
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate(test_ds)

/tmp/ipython-input-1600047660.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aswinraksha2004 (aswinraksha2004-psgitech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.852022,0.593333,0.567770
2,No log,1.479830,0.713333,0.717023


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.852022,0.593333,0.567770
2,No log,1.479830,0.713333,0.717023
3,No log,1.353721,0.786667,0.796970


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.2400091886520386,
 'eval_accuracy': 0.84,
 'eval_f1': 0.8453606699612447,
 'eval_runtime': 43.13,
 'eval_samples_per_second': 3.478,
 'eval_steps_per_second': 0.232,
 'epoch': 3.0}

In [ ]:
trainer.save_model("./bert-spend-cls-final")
tokenizer.save_pretrained("./bert-spend-cls-final")

('./bert-spend-cls-final/tokenizer_config.json',
 './bert-spend-cls-final/special_tokens_map.json',
 './bert-spend-cls-final/vocab.txt',
 './bert-spend-cls-final/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Reload tokenizer and model
model_path = "./bert-spend-cls-final"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
texts = [
    "Appl Inc. IT Eqpt PO-4532 10K",
    "Consulting services - annual review",
    "Flight tickets from New York to Chicago"
]

for t in texts:
    result = classifier(t, truncation=True, padding=True, max_length=128)
    print(f"Input: {t}")
    print(f"Prediction: {result}\n")


Input: Appl Inc. IT Eqpt PO-4532 10K
Prediction: [{'label': 'Software Subscriptions', 'score': 0.1757032722234726}]

Input: Consulting services - annual review
Prediction: [{'label': 'Professional Services > Audit', 'score': 0.2062007337808609}]

Input: Flight tickets from New York to Chicago
Prediction: [{'label': 'Travel > Local Transport', 'score': 0.16044095158576965}]

